- Crear la tabla de races.db
- Importar datos desde races.csv
- Hacer un select * from ...
- Listar 10 primeras registros
- Agrupar por año
- Contar races por año
- Ordenar circuitos de forma decentente

## Importar cosas necesarias

In [ ]:
using Pkg
Pkg.add("SQLite")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("PrettyTables")

## Crear db

In [ ]:
import SQLite

# Create a new SQLite database.
db = SQLite.DB("races.db")

SQLite.execute(db, """
CREATE TABLE IF NOT EXISTS races (
    raceId INTEGER,
    year INTEGER,
    round INTEGER,
    circuitId INTEGER,
    name TEXT,
    date TEXT,
    time TEXT,
    url TEXT
);
""")

println("Database and table created successfully.")


## Importar datos a la db

In [ ]:
using SQLite
using DataFrames
using CSV

# Read CSV file into a DataFrame
df = CSV.read("races.csv", DataFrame)

# Inserty data in tables in SQLite database.
for row in eachrow(df)
    try
        SQLite.execute(db, "INSERT INTO races VALUES (?, ?, ?, ?, ?, ?, ?, ?)", Tuple(row))
    catch e
        println("Error inserting row: ", row)
        println("Error message: ", e)
    end
end

println("Data inserted successfully!")

## Hacer un "SELECT * FROM"

In [ ]:
using SQLite

# load SQLite database
db = SQLite.DB("races.db")

# Query the database to get all data
query = DBInterface.execute(db,"SELECT * FROM races")

# Print all row data in console.
for row in query
    println(row)
end

# Close database connection
SQLite.close(db)

## Listar los primeros 10 registros

In [ ]:
using SQLite
using DataFrames
using PrettyTables

# Load SQLite database
db = SQLite.DB("races.db")

# Query the database to get initial 10 rows
query = DBInterface.execute(db, 
    """
    SELECT * FROM races
    LIMIT 10
    """)
df = DataFrame(query)

# Print the DataFrame in a pretty table format
pretty_table(df)

# Close database connection
SQLite.close(db)


## Agrupar por año

In [ ]:
using SQLite
using DataFrames
using PrettyTables

# Load SQLite database
db = SQLite.DB("races.db")

# Query the database to get all data and sort by year and round
query = DBInterface.execute(db, """
SELECT year, COUNT(circuitId) AS total_circuit
FROM races
GROUP BY year
""")
df = DataFrame(query)

# Print the DataFrame in a pretty table format
pretty_table(df)

# Close database connection
SQLite.close(db)

# Contar races por año

In [ ]:
using SQLite
using DataFrames
using PrettyTables

# Load SQLite database
db = SQLite.DB("races.db")

# Query the database to get all data
query = DBInterface.execute(db, """
SELECT year, COUNT(raceId) AS total_races
FROM races
GROUP BY year
""")
df = DataFrame(query)

# Print the DataFrame in a pretty table format
pretty_table(df)

# Close database connection
SQLite.close(db)

## Ordenar circuitos de forma decentente

In [ ]:
using SQLite
using DataFrames
using PrettyTables

# Load SQLite database
db = SQLite.DB("races.db")

# Query the database order circuitId in descending order
query = DBInterface.execute(db, """
SELECT circuitId, COUNT(raceId) AS total_circuit
FROM races
GROUP BY circuitId
ORDER BY circuitId DESC;
""")
df = DataFrame(query)

# Print the DataFrame in a pretty table format
pretty_table(df)

# Close database connection
SQLite.close(db)

# Ahora según lo pedido en clases por el profesor debemos crear 3 graficos de los datos de la base de datos de lo que nosotros queramos, pero que sean esta db de SQLite.

## Importamos Plots

In [15]:
using Pkg
Pkg.add("Plots")

Creamos el primer grafico en este caso de "cantidad de años en las que se corrio en ese circuito"

In [16]:
using SQLite
using DataFrames
using Plots
using PrettyTables

# Load SQLite database
db = SQLite.DB("races.db")

# Query the database to get year and number of different circuits.
query = DBInterface.execute(db, """
SELECT year, COUNT(DISTINCT circuitId) AS num_circuitos
FROM races
GROUP BY year
ORDER BY year ASC
""")

df = DataFrame(query)

p = Plots.bar(df.year, df.num_circuitos,
    fillcolor = :orange,
    linecolor = :orange,
    title = "Número de circuitos diferentes por año",
    xlabel = "Año",
    ylabel = "Número de circuitos",
    legend = true,
    grid = true,
    label = "Circuitos",
    bar_width = 0.5,
    size = (1500, 700))

# Configurar el eje X para mostrar todos los años
Plots.xticks!(p, xticks, rotation=90)

# Display the graph
display(p)

# Close database connection
SQLite.close(db)

MethodError: MethodError: no method matching xticks!(::Plots.Plot{Plots.GRBackend}, ::typeof(xticks); rotation::Int64)
The function `xticks!` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  xticks!(::Union{Plots.Plot, Plots.Subplot}, !Matched::AbstractVector{T}, !Matched::AbstractVector{S}; kw...) where {T<:Real, S<:AbstractString}
   @ Plots C:\Users\mathi\.julia\packages\Plots\3u4B6\src\shorthands.jl:489
  xticks!(::Union{Plots.Plot, Plots.Subplot}, !Matched::Union{Symbol, Tuple{AbstractVector{T}, AbstractVector{S}}, AbstractVector{T}} where {T<:Real, S<:AbstractString}; kw...)
   @ Plots C:\Users\mathi\.julia\packages\Plots\3u4B6\src\shorthands.jl:487
  xticks!(!Matched::AbstractVector{T}, !Matched::AbstractVector{S}; kw...) where {T<:Real, S<:AbstractString}
   @ Plots C:\Users\mathi\.julia\packages\Plots\3u4B6\src\shorthands.jl:481
  ...


creamos un segundo grafico que cuente los tiempos en cada circuito pero usando el nombre de la pista.

In [ ]:
using SQLite
using DataFrames
using Plots

# Load SQLite database
db = SQLite.DB("races.db")

# Count how many values of time per circuit name
query = DBInterface.execute(db, """
    SELECT name AS circuit_name, COUNT(time) AS time_count
    FROM races
    GROUP BY name
    ORDER BY time_count DESC
""")

df_time = DataFrame(query)

# Create graph
p = bar(df_time.circuit_name, df_time.time_count,
    legend=false,
    title="Cantidad de tiempos registrados por circuito",
    xlabel="Circuito",
    ylabel="Cantidad de tiempos",
    rotation=60,
    size=(1500, 700),
    fillcolor=:orange,
    bar_width=0.5,
    grid=true
)

display(p)

# Close database connection
SQLite.close(db)
